In [ ]:
import torch 
import torchtext.datasets as datasets
import pandas as pd
import numpy as np
#from torchtext.datasets import MultiNLI
import os
import csv
import sys
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from datasets import load_dataset


In [ ]:
# The MNLI dataset can be used to explore decision boundaries and 
# interpretability in natural language inference tasks. By analyzing the decision 
# boundaries of models trained on this dataset, you can gain insights into how models 
# make predictions and understand the relationship between input sentences. 
#The dataset's diverse genres provide a good opportunity to examine how decision boundaries 
# vary across different types of text. Additionally, feature selection techniques can be applied to 
# investigate which features contribute most significantly to the decision boundaries and prediction outcomes.

# Training 

# Evaluation is done using the dev_matched and testing dev_mismatched 

In [ ]:
# Set the field size limit
csv.field_size_limit(sys.maxsize)

def read_jsonl(file_path):
    with open(file_path, "r") as jsonl_file:
        data = [json.loads(line) for line in jsonl_file]
    return data

def read_mnli_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter='\t')
        dataset = [row for row in reader]
    return dataset

def convert_to_dataframe(dataset):
    df = pd.DataFrame(dataset)
    df = df[['sentence1', 'sentence2', 'gold_label']]  # Could include Genre,promptID, and PairID
    return df

# Path to the downloaded JSONL and TXT files
train_file = os.path.expanduser("~/Desktop/Projects/Thesis_Data/MNLI/multinli_1.0_train.txt")

validation_matched_file_txt = os.path.expanduser('~/Desktop/Projects/Thesis_Data/MNLI/multinli_1.0_dev_matched.txt')
validation_matched_file_jsonl = os.path.expanduser('~/Desktop/Projects/Thesis/MNLI/multinli_1.0_dev_matched.jsonl')

test_matched_file_txt = os.path.expanduser('~/Desktop/Projects/Thesis/MNLI/multinli_1.0_dev_mismatched.jsonl')
test_matched_file_jsonl = os.path.expanduser('~/Desktop/Projects/Thesis_Data/MNLI/multinli_1.0_dev_mismatched.txt')



In [ ]:

mnli_dataset = read_mnli_dataset(train_file)
mnli_train_df = convert_to_dataframe(mnli_dataset)

validation_dataset = read_mnli_dataset(validation_matched_file_txt)
validation_df = convert_to_dataframe(validation_dataset)


# Doesn't follow the same format as the validation 
#test_dataset = read_mnli_dataset(test_matched_file_txt)
#test_df = convert_to_dataframe(test_dataset)
#test_df




In [ ]:
mnli_train_df

In [ ]:
validation_df

In [ ]:

mnli_train_df['gold_label'].nunique(), mnli_train_df['gold_label'].value_counts()

In [ ]:

validation_df['gold_label'].nunique(), validation_df['gold_label'].value_counts()

In [ ]:
# Maximmum string length 
mnli_train_df['sentence1'].str.len().max() , validation_df['sentence1'].str.len().max() 

In [ ]:
# Set the desired number of rows for each class
desired_rows_per_class = 1000
# 2 hours 

# Reduce the number of rows for each class
mnli_train_df = mnli_train_df.groupby('gold_label').apply(lambda x: x.sample(desired_rows_per_class, random_state=42)).reset_index(drop=True)
mnli_train_df['gold_label'].nunique(), mnli_train_df['gold_label'].value_counts()

In [ ]:
# Set the desired number of rows for each class
desired_rows_per_class = 200
# 2 hours 

# Reduce the number of rows for each class
validation_df = validation_df[validation_df['gold_label'] != '-']

validation_df = validation_df.groupby('gold_label').apply(lambda x: x.sample(min(desired_rows_per_class, len(x)), random_state=42)).reset_index(drop=True)
validation_df['gold_label'].nunique(), validation_df['gold_label'].value_counts()

In [ ]:
#drop 183
#reduce the size of training 

In [ ]:
# load the dataset
# The root needs to be changed but idk exactly what it does or what it entails

train_data, dev_matched_data, dev_mismatched_data = datasets.MNLI(root='MNLI', split=('train', 'dev_matched', 'dev_mismatched'))

In [ ]:
# Example: Print some examples from the training data


In [ ]:
#stopword removal
#stemming / lemitization

In [ ]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')


In [ ]:
# Use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'Desktop/Projects/Thesis_Data/MNLI/bert-base-uncased'
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Function to remove stopwords from a string
def remove_stopwords(text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = tokenizer.tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Function to apply stemming to a text
def apply_stemming(text):
    tokens = word_tokenize(text)  # Tokenize the text
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Perform stemming
    stemmed_text = ' '.join(stemmed_tokens)  # Join the stemmed tokens back into a string
    return stemmed_text

# Initialize stemmer
stemmer = PorterStemmer()


# Apply stemming to the 'Text' column
mnli_train_df['sentence1'] = mnli_train_df['sentence1'].apply(apply_stemming)
mnli_train_df['sentence2'] = mnli_train_df['sentence2'].apply(apply_stemming)

validation_df['sentence1'] = validation_df['sentence1'].apply(apply_stemming)
validation_df['sentence2'] = validation_df['sentence2'].apply(apply_stemming)

# Apply the remove_stopwords function to the 'Text' column
mnli_train_df['sentence1'] = mnli_train_df['sentence1'].apply(remove_stopwords)
mnli_train_df['sentence2'] = mnli_train_df['sentence2'].apply(remove_stopwords)

validation_df['sentence1'] = validation_df['sentence1'].apply(remove_stopwords)
validation_df['sentence2'] = validation_df['sentence2'].apply(remove_stopwords)


model = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
    num_labels = 3, 
    output_attentions = False, 
    output_hidden_states = False,
)






In [ ]:
def prepare_dataset(df):
    # Tokenize and prepare input data
    sentences = df[['sentence1', 'sentence2']].values.tolist()
    labels = df['gold_label'].apply(lambda label: 0 if label=='entailment' else 1 if label=='neutral' else 2).values

    # Create attention masks
    input_ids, attention_masks = [], []
    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sent,                     
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 100,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                            truncate = True
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0), torch.tensor(labels)



In [ ]:
train_inputs, train_masks, train_labels = prepare_dataset(mnli_train_df)
validation_inputs, validation_masks, validation_labels = prepare_dataset(validation_df)

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=32)


In [ ]:
from torch.utils.data import Dataset, DataLoader

# Let's create a simple dataset
class SimpleDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]



In [ ]:
from transformers import get_linear_schedule_with_warmup

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )

# We'll use number of epochs and the number of training steps as the argument to the scheduler function
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Specify the loss function
loss_fn = torch.nn.CrossEntropyLoss().to(device)

In [ ]:
from tqdm import tqdm

# Training loop
for epoch in range(epochs):
    print(f'Starting epoch {epoch+1}/{epochs}')

    # Training
    model.train()
    total_loss = 0

    # Use tqdm to wrap the train_dataloader for progress monitoring
    train_iterator = tqdm(train_dataloader, desc="Training Iteration", leave=False)

    for step, batch in enumerate(train_iterator):
        # Move the batch to the device
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        # Clear any previously calculated gradients
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]

        # Backward pass
        loss.backward()

        # Clip the norm of the gradients to 1.0
        # Gradient clipping helps prevent "exploding gradients" problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update weights
        optimizer.step()

        # Update the learning rate
        scheduler.step()

        total_loss += loss.item()

        # Update tqdm description with the current loss
        train_iterator.set_description(f"Training Loss: {loss.item():.4f}")

    # Average loss
    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Train loss: {avg_train_loss}')

    # Validation
    model.eval()
    total_loss = 0

    # Use tqdm to wrap the validation_dataloader for progress monitoring
    val_iterator = tqdm(validation_dataloader, desc="Validation Iteration", leave=False)

    for batch in val_iterator:
        # Move the batch to the device
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        # Forward pass
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Get the logits from the outputs tuple: (loss, logits)
        logits = outputs[1]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the loss for this batch of validation sentences and accumulate it over all batches.
        total_loss += loss_fn(torch.tensor(logits), torch.tensor(label_ids)).item()

        # Update tqdm description with the current loss
        val_iterator.set_description(f"Validation Loss: {total_loss:.4f}")

    # Average loss
    avg_val_loss = total_loss / len(validation_dataloader)
    print(f'Validation loss: {avg_val_loss}')


In [ ]:
# Hook function to grab the activation values in the forward pass

activations = []

def forward_hook(module, input, output):
    activations.append(output)

hook_handle = model.bert.embeddings.register_forward_hook(forward_hook)

# Run your model

hook_handle.remove()

# Now `activations` contains the outputs of the module you hooked.


In [ ]:
def randomized_feature_importance(model, X, y, n_iterations=100):
    feature_importances = np.zeros(X.shape[1])
    
    for i in range(n_iterations):
        X_perturbed = X.copy()  # Make a copy of the original features
        
        # Perturb the feature values by adding random noise
        for j in range(X.shape[1]):
            noise = np.random.normal(0, 1, X.shape[0])  # Generate random noise
            X_perturbed[:, j] += noise
        
        # Make predictions with the perturbed features
        y_pred = model.predict(X_perturbed)
        
        # Measure the impact on performance (e.g., accuracy)
        accuracy = accuracy_score(y, y_pred)
        
        # Accumulate the impact scores for each feature
        feature_importances += (accuracy - model.score(X, y)) / n_iterations
    
    return feature_importances


In [ ]:
def permutation_feature_importance(model, X, y, n_iterations=100):
    feature_importances = np.zeros(X.shape[1])
    
    for i in range(n_iterations):
        X_shuffled, y_shuffled = shuffle(X, y)  # Randomly shuffle the data
        
        # Make predictions with the shuffled features
        y_pred = model.predict(X_shuffled)
        
        # Measure the impact on performance (e.g., accuracy)
        accuracy = accuracy_score(y_shuffled, y_pred)
        
        # Accumulate the impact scores for each feature
        feature_importances += (accuracy - model.score(X, y)) / n_iterations
    
    return feature_importances
